# Домашнее задание №3

Тема "Part-of-Speech разметка, NER , извлечение отношений."

Используем предобработанные в рамках 1-ого домашнего задания датасет df.pkl. Используем столбец 'clean_tweet'.


In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import sklearn
import numpy as np
import re

In [2]:
# Загрузим сохраненные данные из предыдущей домашней работы
combine_df = pd.read_pickle("df.pkl")
combine_df.head(3)

,id,label,tweet,clean_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"[when, father, is, dysfunctional, and, is, so,...","[father, dysfunctional, selfish, drags, kids, ...","[father, dysfunct, selfish, drag, kid, dysfunc...","[father, dysfunctional, selfish, drag, kid, dy..."
1,2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use cause they d...,"[thanks, for, lyft, credit, can, not, use, cau...","[thanks, lyft, credit, use, cause, offer, whee...","[thank, lyft, credit, use, caus, offer, wheelc...","[thanks, lyft, credit, use, cause, offer, whee..."
2,3,0.0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]","[bihday, majesty]","[bihday, majesti]","[bihday, majesty]"


## Задание 1

Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)






  




In [3]:
import spacy
from spacy import displacy
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

In [4]:
# Функция подсчета именованных сущностей и их типов
def add_to_dict(dict_, word):
  if word in dict_.keys():
    dict_[word] += 1
  else:
    dict_[word] = 1

In [5]:
# Проверим работу на небольшом массиве текста

text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with recode earlier "
        "this week.")

doc = nlp(text)

# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

# Find named entities, phrases and concepts
for entity in doc.ents:
    print(entity.text, entity.label_)

Noun phrases: ['Sebastian Thrun', 'self-driving cars', 'Google', 'few people', 'the company', 'him', 'I', 'you', 'very senior CEOs', 'major American car companies', 'my hand', 'I', 'Thrun', 'an interview', 'recode']
Verbs: ['start', 'work', 'drive', 'take', 'can', 'tell', 'would', 'shake', 'turn', 'talk', 'say']
Sebastian NORP
Google ORG
2007 DATE
American NORP
earlier this week DATE


In [6]:
# Собираем все твиты в одну строку
text =  " ".join(combine_df['clean_tweet'].values)

In [7]:
ent_dict = {}
ent_types_dict = {}
# Проходим по всем твитам
%time
doc = nlp(text[:1000000])
# Проходим по всем именованным сущностям в text и 
# добавляем в соответствующие словари эти сущности и их типы с подсчетом количества
for entity in doc.ents:
  add_to_dict(ent_dict, entity.text)
  add_to_dict(ent_types_dict, entity.label_)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


In [8]:
# Для удобства преобразуем словари в датафреймы
ent_types_df = pd.DataFrame.from_dict(ent_types_dict, columns=['cnt'], orient='index')
ent_df = pd.DataFrame.from_dict(ent_dict, columns=['cnt'], orient='index')
ent_types_df.sort_values(by='cnt', ascending=False, inplace=True)
ent_df.sort_values(by='cnt', ascending=False, inplace=True)

In [9]:
# ТОП-20 популярных NER в combine_df датасете
ent_df.head(20)

,cnt
today,435
bihday,294
first,168
tomorrow,166
friday,165
one,162
summer,129
sunday,125
tonight,122
saturday,80


In [10]:
# Самый популярный тип NER - DATE
ent_types_df.head(5)

,cnt
DATE,3117
PERSON,2579
GPE,1487
ORG,1030
TIME,569


С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?


In [11]:
# train_df содержал твиты с id от 1 до 31962, а test_df - от 31963 до 49159 включительно
train_df = combine_df.loc[combine_df.id < 31963]
test_df = combine_df.loc[combine_df.id > 31963]

In [12]:
 # Собираем все твиты в одну строку
train =  " ".join(train_df['clean_tweet'].values)
test =  " ".join(test_df['clean_tweet'].values)
train = train[:1000000]
test = test[:1000000]
print(len(train), len(test))

1000000 1000000


In [13]:
train_person_dict = {}
train_org_dict = {}

%time
doc = nlp(train)
# Проходим по всем именованным сущностям в text и добавляем в train_person_dict
# сущности типа PERSON, а в train_org_dict - ORG и подсчитываем количество
for entity in doc.ents:
  if entity.label_ == 'PERSON':
    add_to_dict(train_person_dict, entity.text)
  if entity.label_ == 'ORG':
    add_to_dict(train_org_dict, entity.text)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


In [14]:
# Для удобства преобразуем словари в датафреймы
train_person_df = pd.DataFrame.from_dict(train_person_dict, columns=['cnt'], orient='index')
train_org_df = pd.DataFrame.from_dict(train_org_dict, columns=['cnt'], orient='index')
train_person_df.sort_values(by='cnt', ascending=False, inplace=True)
train_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [15]:
# Топ-20 обсуждаемых персон в train
train_person_df.head(20)

,cnt
selfie,29
hu,23
tbt,17
hillary,15
don,14
christina grimmie,10
brexit,9
ripchristina,9
stas,9
yo,8


In [16]:
# Топ-20 обсуждаемых организаций в train
train_org_df.head(20)

,cnt
tbt,26
gop,18
nba,17
aap,14
fed,13
islam,11
sma,9
nd,9
eu,9
apple,8


In [17]:
test_person_dict = {}
test_org_dict = {}
# Проходим по всем твитам
%time
doc = nlp(test)
# Проходим по всем именованным сущностям в каждом твите и добавляем в test_person_dict
# сущности типа PERSON, а в test_org_dict - ORG и подсчитываем количество
for entity in doc.ents:
  if entity.label_ == 'PERSON':
    add_to_dict(test_person_dict, entity.text)
  if entity.label_ == 'ORG':
    add_to_dict(test_org_dict, entity.text)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.91 µs


In [18]:
# Для удобства преобразуем словари в датафреймы
test_person_df = pd.DataFrame.from_dict(test_person_dict, columns=['cnt'], orient='index')
test_org_df = pd.DataFrame.from_dict(test_org_dict, columns=['cnt'], orient='index')
test_person_df.sort_values(by='cnt', ascending=False, inplace=True)
test_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [19]:
# Топ-20 обсуждаемых персон в test
test_person_df.head(20)

,cnt
selfie,40
hu,30
tbt,30
trump,20
yo,13
xxx,12
don,11
christina grimmie,10
jesus,10
regrann,10


In [20]:
# Топ-20 обсуждаемых организаций в test
test_org_df.head(20)

,cnt
tbt,22
gop,19
nba,15
aap,9
nd,9
nike,8
cnn,8
amazon,8
na,8
nra,7


Видим, что в топах присутствует мусор

## Задание 2

Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.


In [21]:
import nltk
from nltk.corpus import names
# Устанавливаем необходимые ресурсы
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('maxent_ne_chunker') 
nltk.download('words')  

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [22]:
# Проверим работу на небольшом массиве текста

text = ("When Sebastian Thrun started working on self-driving cars at "
        "Google in 2007, few people outside of the company took him "
        "seriously. “I can tell you very senior CEOs of major American "
        "car companies would shake my hand and turn away because I wasn’t "
        "worth talking to,” said Thrun, in an interview with recode earlier "
        "this week.")

for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))):
  if hasattr(chunk, 'label'):
    print((' '.join(c[0] for c in chunk), chunk.label()))

('Sebastian Thrun', 'PERSON')
('Google', 'ORGANIZATION')
('CEOs', 'ORGANIZATION')
('American', 'GPE')
('Thrun', 'PERSON')


In [23]:
# Собираем все твиты в одну строку
text =  " ".join(combine_df['clean_tweet'].values)
text = text[:1000000]

In [24]:
# Так как nltk обнаруживает именованные сущности только если они написаны с заглавной буквы, 
# нужно первые буквы всех слов в text перевести в верхнй регистр
text = text.title()

In [25]:
ent_nltk_dict = {}
ent_nltk_types_dict = {}

for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(text))):
  if hasattr(chunk, 'label'):
    add_to_dict(ent_nltk_dict, (' '.join(c[0] for c in chunk)))
    add_to_dict(ent_nltk_types_dict, chunk.label())

In [28]:
# Для удобства преобразуем словари в датафреймы
ent_nltk_types_df = pd.DataFrame.from_dict(ent_nltk_types_dict, columns=['cnt'], orient='index')
ent_nltk_df = pd.DataFrame.from_dict(ent_nltk_dict, columns=['cnt'], orient='index')
ent_nltk_types_df.sort_values(by='cnt', ascending=False, inplace=True)
ent_nltk_df.sort_values(by='cnt', ascending=False, inplace=True)

In [29]:
# ТОП-20 популярных NER в combine_df датасете
ent_nltk_df.head(20)

,cnt
Yoyour,145
Yoyou,141
Will,100
New,74
Us,53
Delete Old,43
Happy,43
Orlando,42
Forward,35
Libtard,33


In [30]:
# Самый популярный тип NER - DATE
ent_nltk_types_df.head(5)

,cnt
PERSON,12208
ORGANIZATION,5640
GPE,1841
FACILITY,441
LOCATION,28


С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

In [31]:
# Так как nltk обнаруживает именованные сущности только если они написаны с заглавной буквы, 
# нужно первые буквы всех слов в train и test перевести в верхнй регистр
train_nltk = train.title()
test_nltk = test.title()

In [32]:
train_person_dict = {}
train_org_dict = {}

for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(train_nltk))):
  if hasattr(chunk, 'label'):
    if chunk.label() == 'PERSON':
      add_to_dict(train_person_dict, (' '.join(c[0] for c in chunk)))
    if chunk.label() == 'ORGANIZATION':
      add_to_dict(train_org_dict, (' '.join(c[0] for c in chunk)))

In [33]:
# Для удобства преобразуем словари в датафреймы
train_person_df = pd.DataFrame.from_dict(train_person_dict, columns=['cnt'], orient='index')
train_org_df = pd.DataFrame.from_dict(train_org_dict, columns=['cnt'], orient='index')
train_person_df.sort_values(by='cnt', ascending=False, inplace=True)
train_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [34]:
# Топ-20 обсуждаемых персон в train_lntk
train_person_df.head(20)

,cnt
Yoyou,139
Will,100
Delete Old,43
Happy,38
Forward,35
Libtard,33
Father,31
Grateful,27
Been,27
Sad,26


In [35]:
# Топ-20 обсуждаемых организаций в text_lntk
train_org_df.head(20)

,cnt
Vicinity Of Their,25
Fathers,21
Lot,20
Well Deletetweets,18
Vast Expanse Of Mountains,18
House,16
All,14
Work Conference,14
Same,12
No,11


In [36]:
test_person_dict = {}
test_org_dict = {}

for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(test_nltk))):
  if hasattr(chunk, 'label'):
    if chunk.label() == 'PERSON':
      add_to_dict(test_person_dict, (' '.join(c[0] for c in chunk)))
    if chunk.label() == 'ORGANIZATION':
      add_to_dict(test_org_dict, (' '.join(c[0] for c in chunk)))

In [37]:
# Для удобства преобразуем словари в датафреймы
test_person_df = pd.DataFrame.from_dict(test_person_dict, columns=['cnt'], orient='index')
test_org_df = pd.DataFrame.from_dict(test_org_dict, columns=['cnt'], orient='index')
test_person_df.sort_values(by='cnt', ascending=False, inplace=True)
test_org_df.sort_values(by='cnt', ascending=False, inplace=True)

In [38]:
# Топ-20 обсуждаемых персон в test_lntk
test_person_df.head(20)

,cnt
Yoyou,132
Will,97
Happy,43
Forward,40
Delete Old,34
Father,32
Ready,26
Libtard,24
Grateful,23
Good,22


In [39]:
# Топ-20 обсуждаемых организаций в test_lntk
test_org_df.head(20)

,cnt
Vicinity Of Their,23
Vast Expanse Of Mountains,22
Lot,21
Happiness,20
All,19
Best,18
Same,15
House,15
Work Conference,15
Only,13


## Задание 3

Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.

Лучше отработала библиотека SPACY. NLTK распознала много ложных сущностей из-за приведения первой буквы каждого слова в верхний регистр. Но без этого она вообще не выявляет ни одной сущности.